# **BUSI 488 / COMP 488 Data Science in the Business World**
## *Spring 2023*  
Daniel M. Ringel  
Kenan-Flagler Business School  
*The University of North Carolina at Chapel Hill*  
dmr@unc.edu  

---
## Missing Data Examples for Class 03
*January 17th, 2023*  
Version 1.2  


----

# 0. Mount your drive where the dataset is located

1. Run Cell below
2. Click on link ***Go to this URL in a browser:***
3. Login to your Google Drive in the new browser tab and allow access
4. OPTIONAL: Copy the generated authorization code (if you get one)
5. OPTIONAL: Paste the authorization code in the provided field ***Enter your authorization code:*** (if it is shown - not always the case)
6. Hit ENTER
7. You might also be promted to verify access through a code via text message

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Let's navigate to the folder on your Google Drive that you uploaded the files for this notebook to:  
- "MoviesOnStreamingPlatforms_updated.csv"
- "complaints-credit-debit-cards+fees-interest-2019-2020.csv"  

1. %cd is a command that means "change directory". In other words: go to the following folder
2. following %cd, you need to provide the "path" (i.e., hierachie of folders) to the folder where your data file(s) are.   
  
**Warning:** Your path does not include your file! Only the folder it is contained in!

3. keep the first part /content/gdrive/MyDrive/ the same as in the code cell below
4. change the part /Teaching/2023/488/CoLab_Tutorial to your own folder structure (i.e., the folders you created on your google drive)
   - for example, if you created a folder in your google drive called 488, 
   - and within that folder (488) you created another folder called Class03,
   - then you need to write the following: %cd /content/gdrive/MyDrive/488/Class03
   - DO NOT include the filename of the file you want to ultimately load inm your path!
5. make sure that the files you want to load are actually in that folder, that is, you uploaded them there (on your google drive)
6. to upload the files from Canvas to Google Drive, you first need to download them to your computer from Canvas and then upload them to Google Drive (unless you are automatically syncing your computer to the Google Cloud)

In [ ]:
# Let's assume you've created a folder called "488" on your Google drive for the course and a folder inside the "488" folder for class 3 called "Class03"
# You need to upload the files for this class (this .ipynb notbooks as well as the two CSV files) that you downloaded from Canvas to that folder.

# Now that you Google Drive is connected, you can navigate to that folder with the %cd command as done below:
%cd /content/gdrive/MyDrive/488/Class03

# To see waht files are in the folder, you can use a special shell command (!ls) to view the files in the home directory of the notebook environment
!ls 

/content/gdrive/MyDrive/488/Class03
488_2023_Class03_MissingData_20230117.ipynb
complaints-credit-debit-cards+fees-interest-2019-2020.csv
MoviesOnStreamingPlatforms_updated.csv


# 1. Explore Missing Data
We will study missing data in a large data set about movies on streaming platforms. The data were scraped in 2020 from various platforms and merged with IMDb data (https://www.imdb.com/).    

#### **Where to get the Data**  

You can download the data set for this notebook from Kaggle:  
https://www.kaggle.com/ruchi798/movies-on-netflix-prime-video-hulu-and-disney

## 1.1 Read data into a DataFrame

In [ ]:
import pandas as pd
df = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")
print(df.shape)
df.head()

## 1.2 Check for Missing Data: Summary

In [ ]:
df.isna().sum()

It looks like we have some missing data. Most of the missing data relate to "***Rotten Tomatoes***".  

***Rotten Tomatoes*** score measures the percentage that are more positive than negative, and assigns an overall fresh or rotten rating to the movie. Scores of over 60 percent are considered fresh, and scores of 59 percent and under are rotten.   
https://www.vox.com/culture/2017/8/31/16107948/rotten-tomatoes-score-get-their-ratings-top-critics-certified-fresh-aggregate-mean

## 1.3 Visually inspect Missing Data
Sometimes it is easier to see what is going on in your data when you visualize them.   

The **Missingno library** offers a very nice way to visualize the distribution of NaN values.   

Missingno is a Python library and compatible with Pandas. 

While missingno is available on CoLab, it may not be installed on your computer. If you want to use it, you may fist need to go to your Terminal and run: `pip install missingno`  

See the github repository to learn more about missingno:  
https://github.com/ResidentMario/missingno

In [ ]:
import missingno as msno
%matplotlib inline

In [ ]:
msno.matrix(df)

It looks like there is quite a lot of missing data (white areas in the Figure).  

The sparkline at right summarizes the general shape of the data completeness and points out:
- the rows with the maximum 
- the rows with the minimum     

nullity in the dataset.

## 1.4 Systematically Missing?
Take a closer look - are missing values related to another? 

In [ ]:
msno.heatmap(df)

The heatmap highlights positive correlation by the level level of darkness in blue (negative correlation would be red).   

- Various levels of positive correlation exist between “Directors”, “Genres”, “Country”, “Language”, and “Runtime”
- The greatest positive correlation is between “Language” and “Country” (0.8)
- Apparently, there is a relationship between missing data.  

***Ask yourself:***
1. What do you think the implications are? 
2. What can we learn from these missing data?
3. Can we impute these missing data? 
4. Should we impute them? Why or why not?
5. Should we drop observations with missing data? Why or why not?

# 2. Impute Missing Data in Time Series

We will look at a time series of consumer complaints about credit and debit cards.  

To illustrate the imputation of time series data:
1. We will create missing data in the series and 
2. then try to fill them again with various imputation techniques.

### Where to get the data: 
You can download them directly from the *Consumer Financial Protection Bureau (CFPB):  
https://www.consumerfinance.gov/data-research/consumer-complaints/search/?chartType=line&dateInterval=Week&date_received_max=2020-12-31&date_received_min=2019-01-01&issue=Fees%20or%20interest&lens=Overview&product=Credit%20card%20or%20prepaid%20card&searchField=all&tab=Trends

or  

you can find the file ***complaints-credit-debit-cards+fees-interest-2019-2020.csv*** on CANVAS along with this notebook.

## 2.1 Import Data and Prepare it

In [ ]:
import pandas as pd

# read csv file
df = pd.read_csv("complaints-credit-debit-cards+fees-interest-2019-2020.csv")

# show columns
display(df.columns.tolist())

# show shape and the first 5 rows
print(df.shape)
df.head()


In [ ]:
# let's see which format the data is in
df.info()

In [ ]:
# Need to convert "Date received" to a pandas date time format
df['Date received']= pd.to_datetime(df['Date received'])

# check that it worked
df.info()
display(df['Date received'].head())

In [ ]:
# Let's sort the data by date and check the range
df.sort_values(by=['Date received'], inplace=True)
display(df['Date received'].min())
display(df['Date received'].max())

In [ ]:
# We  aggregate the number of complaints at the weekly level
weeklyOriginal=df[['Date received', 'Complaint ID']].groupby(pd.Grouper(key='Date received',freq='W')).count()

# Let's rename the aggregated ID column to "NumComplaints"
weeklyOriginal = weeklyOriginal.rename(columns={'Complaint ID': 'NumComplaints'})

# Take a look
weeklyOriginal.head()

## 2.2 Remove Observations

We are going to create some missing data for demonstration purposes. We will then try to recover them using imputation techniques.

In [ ]:
# Create a second dataframe
weekly=df[['Date received', 'Complaint ID']].groupby(pd.Grouper(key='Date received',freq='W')).count()
# Let's rename the aggregated ID column to "NumComplaints"
weekly = weeklyOriginal.rename(columns={'Complaint ID': 'NumComplaints'})

# We set the complaint counts for select date ranges to NAN
import numpy as np
weekly[(weekly.index >= pd.to_datetime("2019-01-14")) & (weekly.index <= pd.to_datetime("2019-02-10"))] = np.nan
weekly[(weekly.index >= pd.to_datetime("2019-04-08")) & (weekly.index <= pd.to_datetime("2019-04-28"))] = np.nan
weekly[(weekly.index >= pd.to_datetime("2019-06-24")) & (weekly.index <= pd.to_datetime("2019-08-25"))] = np.nan
weekly[(weekly.index >= pd.to_datetime("2019-09-23")) & (weekly.index <= pd.to_datetime("2019-10-27"))] = np.nan
weekly[(weekly.index >= pd.to_datetime("2020-01-20")) & (weekly.index <= pd.to_datetime("2020-02-24"))] = np.nan
weekly[(weekly.index >= pd.to_datetime("2020-09-17")) & (weekly.index <= pd.to_datetime("2020-11-22"))] = np.nan
display(weekly.info())

## 2.3 Visualize our Time Series

In [ ]:
# Missing Data
weeklyOriginal['NumComplaints'].plot(color='lightgrey', marker='o', linestyle='dotted', figsize=(30, 10))
weekly['NumComplaints'].plot(title='Weekly Complaints', marker='o', figsize=(30, 10))

## 2.4 Impute Missing Data and Visualize

In [ ]:
# Forward Fill
Forward = weekly.fillna(method='ffill')
weeklyOriginal['NumComplaints'].plot(color='lightgrey', marker='o', linestyle='dotted', figsize=(30, 10))
Forward['NumComplaints'].plot(color='red', marker='o', linestyle='dotted')
weekly['NumComplaints'].plot(title='Weekly Complaints', marker='o')

# check correlation
display(weeklyOriginal['NumComplaints'].corr(Forward['NumComplaints']))

# Correlation dataframe
display(pd.concat([weeklyOriginal, Forward], axis=1, keys=['weeklyOriginal', 'Forward']).corr().loc['Forward', 'weeklyOriginal'])

In [ ]:
# Back Fill
Backward = weekly.fillna(method='bfill')
weeklyOriginal['NumComplaints'].plot(color='lightgrey', marker='o', linestyle='dotted', figsize=(30, 10))
Backward['NumComplaints'].plot(color='red', marker='o', linestyle='dotted')
weekly['NumComplaints'].plot(title='Weekly Complaints', marker='o')

# check correlation
display(weeklyOriginal['NumComplaints'].corr(Backward['NumComplaints']))

# Correlation dataframe
display(pd.concat([weeklyOriginal, Backward], axis=1, keys=['weeklyOriginal', 'Backward']).corr().loc['Backward', 'weeklyOriginal'])

### Notice that we repeat code in the above cells.   

While it is easy and tempting to just copy/paste code, you will later want to make modifications, and your analysis will get out of sync.  Is there a way to avoid this?   

**Yes, by using functions!**

In the DataCamp assignments, we can learn how to write our own function. Let's refactor our code and organize it into a function.   

In [ ]:
def impute_fill(impute_method, type):
  dt = weekly.fillna(method=impute_method)
  weeklyOriginal['NumComplaints'].plot(color='lightgrey', marker='o', linestyle='dotted', figsize=(30, 10))
  dt['NumComplaints'].plot(color='red', marker='o', linestyle='dotted')
  weekly['NumComplaints'].plot(title='Weekly Complaints', marker='o')
  display(weeklyOriginal['NumComplaints'].corr(dt['NumComplaints']))
  #CorrelationTable: display(pd.concat([weeklyOriginal, dt], axis=1, keys=['weeklyOriginal', type]).corr().loc[type, 'weeklyOriginal'])

In [ ]:
impute_fill('ffill', 'Forward')

In [ ]:
impute_fill('bfill', 'Backward')

## 2.5 Refactoring to do interpolation

There are other imputation methods that interpolate rather than fill, but we can't just reuse the function we defined above.  This is because the function is doing more than one thing 
* it assumes it is working on weekly data
* it does fill interpolation
* and then it displays results.  

**When refactoring, make each function do one thing, and do that well.**

Let's focus here on the plot. 
-  What we do is show three time series layers.
-  From bottom to top, these are:
  -  ground truth, 
  - the imputed, 
  - and the given time series. 
- We also display the correlation coefficient.  That description is both specific enough, and general enough, to be a function by itself. 

In [ ]:
def plot3timelines(bottom, mid, top, mycolumn, mytitle):
  ''' This function plots time series from three dataframes with a common column '''
  bottom[mycolumn].plot(color='lightgrey', marker='o', linestyle='dotted', figsize=(30, 10))
  mid[mycolumn].plot(color='red', marker='o', linestyle='dotted')
  top[mycolumn].plot(title=mytitle, marker='o')
  display(bottom[mycolumn].corr(mid[mycolumn]))


In [ ]:
# Forward Fill 
plot3timelines(weeklyOriginal, weekly.fillna(method='ffill'), weekly, 'NumComplaints', 'Weekly Complaints : Forward Fill')

In [ ]:
# Back Fill 
plot3timelines(weeklyOriginal, weekly.fillna(method='bfill'), weekly, 'NumComplaints', 'Weekly Complaints : Backward Fill')

In [ ]:
# Linear Interpolation
plot3timelines(weeklyOriginal, weekly.interpolate(method='linear'), weekly, 'NumComplaints', 'Weekly Complaints : Linear')

In [ ]:
# Quadratic Interpolation
plot3timelines(weeklyOriginal, weekly.interpolate(method='quadratic'), weekly, 'NumComplaints', 'Weekly Complaints : Quadratic')

In [ ]:
# Nearest Neighbor Interpolation
plot3timelines(weeklyOriginal, weekly.interpolate(method='nearest'), weekly, 'NumComplaints', 'Weekly Complaints : Nearest')

# 3. Tutorials and closing
Check out this nice tutorial on methods in pandas to handle missing data:  
https://towardsdatascience.com/handling-missing-values-with-pandas-b876bf6f008f

The book, [Refactoring: improving the design of existing code](https://www.amazon.com/Refactoring-Improving-Existing-Addison-Wesley-Signature/dp/0134757599/ref=pd_lpo_14_t_0/144-9689246-2900853?_encoding=UTF8&pd_rd_i=0134757599&pd_rd_r=a09d615e-0fd1-4035-b56b-0d3e365a93af&pd_rd_w=4n3e6&pd_rd_wg=CDnNg&pf_rd_p=16b28406-aa34-451d-8a2e-b3930ada000c&pf_rd_r=EBHXKXGTW44N18AZXPCG&psc=1&refRID=EBHXKXGTW44N18AZXPCG), Fowler, is a classic.  Here are two summaries: https://github.com/HugoMatilla/Refactoring-Summary https://www.benlinders.com/2019/summary-of-refactoring-in-15-tweets/ 

## Never forget

- being able to handle missing data is great
- ***BUT*** it does not mean that what you are doing with/to them is appropriate 
- for the problem you are trying to solve